In [1]:
# -*- coding: utf-8 -*-
from keras.preprocessing.image import img_to_array, load_img
from tensorflow.python.keras.models import load_model
from PIL import Image,ImageTk, ImageOps
from keras.models import load_model
from PIL import Image, ImageTk
from tkinter import messagebox
from tkinter import filedialog
import tkinter.filedialog
from pathlib import Path
import mediapipe as mp
import tkinter as tk
import numpy as np
import cv2
import os

        # アプリ情報（キャンバス情報、画面サイズ等）
class Application(tk.Frame):
    def __init__(self,master, video_source=0):
        super().__init__(master)

        self.master.geometry("380x170")
        self.master.minsize(width = 380, height = 170)
        self.master.maxsize(width = 380, height = 170)
        self.master.title("指文字翻訳アプリ")
        
        # テキストボックスの Stringvar の用意
        self.trans_var = tk.StringVar()
        self.casca_var = tk.StringVar()
        self.model_var = tk.StringVar()
        self.camera_var = tk.StringVar()
        
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=1)
        self.columnconfigure(2, weight=1)

        self.create_widgets()
    
        self.delay = 15 #[mili seconds]

        # 画面情報（ボタン表示、キャンバス表示）
    def create_widgets(self):
        
        # 各ボタンの横幅
        but_width = 10
        # 各要素ごとの間隔
        space = 5
        
        # 操作の場所
        self.frame_btn = tk.LabelFrame(self.master)
        self.frame_btn.place(x=10, y=10)
        self.frame_btn.configure(width = 360, height = 150)
        self.frame_btn.grid_propagate(0)

        # 閉じるボタン
        self.can_clo = tk.Button(self.frame_btn, text = 'モデル選択')
        self.can_clo.configure(width = but_width,
                               height = 1,
                               command = self.select_model)
        self.can_clo.grid(column = 2,
                          row = 2,
                          padx = 20,
                          pady = space)
        
        # 画像ボタン
        self.can_clo = tk.Button(self.frame_btn, text = '分類器選択')
        self.can_clo.configure(width = but_width,
                               height = 1,
                               command = self.select_cascade)
        self.can_clo.grid(column = 2,
                          row = 1,
                          pady = space)

        # 動画ボタン
        self.start = tk.Button(self.frame_btn, text = '起動')
        self.start.configure(width = but_width,
                             height = 1,
                             command = self.botton)
        self.start.grid(column = 2,
                        row = 0,
                        pady = space)
        
        # テキスト消去
        del_button = tk.Button(self.frame_btn, text = '消しゴム')
        del_button.configure(width = but_width,
                             height = 1,
                             command = self.btn_clear_click)
        del_button.grid(column = 2,
                        pady = space)
        
        padding = {'padx': 5, 'pady': 5}
        
        # 「翻訳」の場所指定
        tk.Label(self.frame_btn, text = '翻訳').grid(column = 0, row = 0, **padding)
        tk.Label(self.frame_btn, text = '分類器').grid(column = 0, row = 1, **padding)
        tk.Label(self.frame_btn, text = 'モデル').grid(column = 0, row = 2, **padding)
        tk.Label(self.frame_btn, text = 'カメラ').grid(column = 0, row = 3, **padding)
     
        # テキストボックスの設定
        # 翻訳
        self.trans_entry = tk.Entry(self.frame_btn, textvariable = self.trans_var)
        self.trans_entry.configure(font = ("Courier", 11))
        self.trans_entry.grid(column = 1, row = 0, **padding)
        self.trans_entry.focus()
        
        # 分類器
        self.casca_entry = tk.Entry(self.frame_btn, textvariable = self.casca_var)
        self.casca_entry.grid(column = 1, row = 1, **padding)
        self.casca_entry.focus()
        
        # モデル
        self.model_entry = tk.Entry(self.frame_btn, textvariable = self.model_var)
        self.model_entry.grid(column = 1, row = 2, **padding)
        self.model_entry.focus()
        
        # カメラ
        self.camera_entry = tk.Entry(self.frame_btn, textvariable = self.camera_var)
        self.camera_entry.grid(column = 1, row = 3, **padding)
        self.camera_entry.focus()
        
    # 翻訳テキストボックスを空白にする
    def btn_clear_click(self):
        ''' Entryのクリア '''
        self.trans_var.set("")
        

    # 分類器選択
    def select_cascade(self, event = None):
        filename = filedialog.askopenfilename(title = "カスケード分類器を選択",
            filetypes = [("", "*.xml")], # ファイルフィルタ
            initialdir = os.path.abspath(Path().resolve()) # 自分自身のディレクトリ
        )
        self.casca_var.set(filename)
        
    # モデル選択
    def select_model(self, event = None):
        filename = filedialog.askopenfilename(title = "モデルファイルを選択",
            filetypes = [("", "*.json .pickle .pkl .hdf5 .h5")], # ファイルフィルタ
            initialdir = os.path.abspath(Path().resolve()) # 自分自身のディレクトリ
        )
        self.model_var.set(filename)
        
    # 分類器パスが空白かどうかの確認
    def cascade_check(self, CasPath, camera):
        
        if CasPath == "":
            messagebox.showerror('エラー', 'カスケードファイルが選択されていません')
            camera.release()
        elif os.path.exists(CasPath) == False:
            messagebox.showerror('エラー', 'カスケードファイルが見つかりません')
            camera.release()
        else:
            pass
            
    # モデルパスが空白かどうかの確認
    def model_check(self, ModPath, camera):
        
        if ModPath == "":
            messagebox.showerror('エラー', 'モデルが選択されていません')
            camera.release()
        elif os.path.exists(ModPath) == False:
            messagebox.showerror('エラー', 'モデルが見つかりません')
            camera.release()
        else:
            pass
    
    # カメラが指定されているかどうかの確認
    def camera_check(self, CamNum):
        
        if CamNum == "":
            messagebox.showerror('エラー', 'カメラが指定されていません')
        elif CamNum.isdecimal() == False:
            messagebox.showerror('エラー', 'カメラが見つかりません')
        else:
            camera = cv2.VideoCapture(int(CamNum))
        return camera
        
        
    # メイン処理
    def botton(self):
        
        # 変数の準備
        array = []
        hoge = ""
        fuga = ""
        
        # テキストボックスの値を取得
        cas_get = self.casca_entry.get()
        mod_get = self.model_entry.get()
        cam_get = self.camera_entry.get()
        
        camera = self.camera_check(cam_get)
        
        self.cascade_check(cas_get, camera)
        self.model_check(mod_get, camera)
        
        hands = mp.solutions.hands.Hands(
            static_image_mode = False,
            max_num_hands = 2,
            min_detection_confidence = 0.9,
            min_tracking_confidence = 0.9)
        
        
        cascade = cv2.CascadeClassifier(cas_get)
        model = load_model(mod_get)
        
        while camera.isOpened():

            ret, frame = camera.read()
            
            frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
            results = hands.process(frame)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # 手のポイントの準備
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:

                    mp.solutions.drawing_utils.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp.solutions.hands.HAND_CONNECTIONS,
                        mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                        mp.solutions.drawing_styles.get_default_hand_connections_style()
                    )

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face = cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 3, minSize = (90, 90))
            for (x,y,w,h) in face:
                cv2.rectangle(frame,(x, y),(x + 130, y + 130),(0,0,200),3)
                frame_cut = frame[y + 5:y + 205,x + 5:x + 205]
                
                img = img_to_array(frame_cut)
                img_nad = img_to_array(img)/255
                img_nad = img_nad[None, ...]

                label=['a','thi','thu']
                
                try:
                    pred = model.predict(img_nad, batch_size=1, verbose=0)
                except:
                    continue
                    
                score = np.max(pred)
                pred_label = label[np.argmax(pred[0])]
                
                if hoge != pred_label:
                    
                    hoge = pred_label
                    array.clear()
                    array.append(pred_label)
                    cv2.putText(frame,pred_label,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255), thickness=2)
                else:
                    array.append(pred_label)
                    cv2.putText(frame,pred_label,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255), thickness=2)

                    if len(array) >= 15: 
                        self.trans_entry.insert(tkinter.END, pred_label)
                        fuga += pred_label
                        array.clear()
                        
            cv2.putText(frame, fuga, (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), thickness=2)
            cv2.imshow('camera', frame) 

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        camera.release()
        cv2.destroyAllWindows()     

# GUIのスタート
def main():
    root = tk.Tk()
    app = Application(master = root)#Inherit
    app.mainloop()

if __name__ == "__main__":
    main()